In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import sys
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from neural_ode import (
    LatentODEfunc,
    RecognitionRNN,
    Decoder,
    VAE,
    train,
    vae_loss,
) 
from torch.optim import Adam
import elephant

sys.path.append("/home/phornauer/Git/DeePredictions/data_processing")
import preprocessing as pp

/home/phornauer/miniconda3/envs/si_env/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics/Large_dose_range/well014/sorter_output/"
bl_times, bl_templates = pp.load_sorting_results(root_path)

drug_times, drug_templates = pp.load_sorting_results(root_path,segment_idx=3)

bl_spike_trains = pp.generate_unit_spike_train_list(bl_times, bl_templates)
unit_ids = np.unique(bl_templates)
drug_spike_trains = pp.generate_unit_spike_train_list(drug_times, drug_templates,unit_ids=unit_ids)
sig_idx, fr_change = pp.find_significant_firing_rate_changes(root_path,post_idx=5,side="right")
bl_spike_trains_sig = elephant.neo_tools.SpikeTrainList([st for i, st in enumerate(bl_spike_trains) if sig_idx[i]])
bl_spike_trains_non_sig = elephant.neo_tools.SpikeTrainList([st for i, st in enumerate(bl_spike_trains) if not sig_idx[i]])
firing_rates = pp.generate_population_activities(bl_spike_trains,sig_idx, bin_size=0.005)

In [34]:
data = firing_rates
data_tensor = torch.tensor(data.transpose(), dtype=torch.float32)  # Convert to tensor and transpose to (N, 2)
n_samples = 500
n_batches = data_tensor.shape[0] // n_samples
data_tensor = data_tensor[:n_batches*n_samples]  # Discard the remainder
data_tensor = data_tensor.reshape((n_batches, n_samples, data.shape[0]))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the dimensions
hidden_dim = 50
latent_dim = 20
obs_dim = data.shape[0]
print(data_tensor.shape)


torch.Size([480, 500, 2])


In [55]:
func = LatentODEfunc(latent_dim, hidden_dim).to(device)
rec = RecognitionRNN(latent_dim, obs_dim, hidden_dim).to(device)
dec = Decoder(latent_dim, obs_dim, hidden_dim).to(device)
params = (list(func.parameters()) + list(dec.parameters()) + list(rec.parameters()))

model = VAE(func, rec, dec, obs_dim, latent_dim).to(device)

dataset = TensorDataset(data_tensor)  # Create a TensorDataset
data_loader = DataLoader(dataset, batch_size=32)  # Create a DataLoader

optimizer = torch.optim.Adam(params, lr=0.001)

losses = []
for epoch in range(100):  # Train for 100 epochs
    epoch_loss = train(model, data_loader, device, optimizer, num_epochs=1)
    losses.append(epoch_loss)

# Save the model
torch.save(model.state_dict(), 'model.pth')

# Save the training progress
np.savetxt('training_progress.csv', losses, delimiter=',')

# Visualize the training progress
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training progress')
plt.show()

torch.Size([32, 500, 2])
torch.Size([500, 500, 2])


/home/phornauer/Git/DeePredictions/model/neural_ode.py:109: UserWarning: Using a target size (torch.Size([32, 500, 2])) that is different to the input size (torch.Size([500, 500, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  recon_loss = F.mse_loss(x_hat, x, reduction="sum")


RuntimeError: The size of tensor a (500) must match the size of tensor b (32) at non-singleton dimension 0

In [41]:
xd = torch.Tensor([1,2,3]).to(device)

In [42]:
xd.device

device(type='cuda', index=0)